In [1]:
import numpy as np
import pandas as pd
from sklearn import (model_selection, 
                     linear_model, 
                     discriminant_analysis, 
                     neighbors, 
                     tree, 
                     naive_bayes, 
                     metrics,
                     svm,
                     ensemble,
                     preprocessing)
import pickle
from matplotlib import pyplot
import itertools
import math
import warnings
from IPython.display import display
warnings.filterwarnings('ignore')

## Loading Flattened Image Data

In [2]:
inStream = open('flattened_image_RGB_data_metadata.pickle', 'rb')
data, metadata = pickle.load(inStream)
inStream.close()

## Some more image prep

In [3]:
imgData=pd.DataFrame(data)
imgData["gender"]=metadata["gender"]
imgData.shape

(4523, 10001)

In [4]:
my_features_Column_labels=data.columns[:-1]
my_target_Column_label="gender"

## Learning functions

In [5]:
def normalizeConfusionMatrix(confusionMatrix):
    normConfusionMatrix = confusionMatrix.astype('float') / confusionMatrix.sum(axis=1)[:, np.newaxis]
    return normConfusionMatrix

In [6]:
def train(df, featureColLabels, targetColLabel, model, train_size=0.5, 
          random_state=2,shuffle=False, return_all=False):
    fts = df[featureColLabels]
    features = fts.values

    tgs = df[targetColLabel]
    target = tgs.values 

    (features_train, features_test, target_train, target_test) = model_selection.train_test_split(
    features, target, train_size=train_size, random_state=random_state,shuffle=shuffle)

    trainedModel=model.fit(features_train, target_train)
    if return_all is True:
        return trainedModel, features_train, features_test, target_train, target_test
    elif return_all is False:
        return trainedModel, features_test, target_test
    else:
        return None

def test(features_test, target_test, trainedModel, normalize_cm=True):
    predictions = trainedModel.predict(features_test)
    accuracy=metrics.accuracy_score(target_test, predictions)
    confusionMatrix=metrics.confusion_matrix(target_test, predictions)
    precision=metrics.precision_score(target_test,predictions, average=None)
    if normalize_cm==True:
        # normalize the confusion matrix.
        confusionMatrix=normalizeConfusionMatrix(confusionMatrix)
    return (accuracy, precision, confusionMatrix)

In [7]:
def trainAndTest(df, featureColLabels, targetColLabel, model, train_size=0.5, 
                     random_state=2,shuffle=False, normalize_cm=True, return_all=False):
        '''
        ARGs:
            >> df: a pandas DataFrame of the dataset.
            >> featureColLabels: a list of strings that represents the colunm name of the features.
            >> targetColLabel: a string that represents the colunm name of the target.
            >> model: an sklearn model constructor.
        OPs:
            Trains and tests the dataset using the specified model.
        RETURN:
            >> If return_all is not a boolean: it returns None.
            >> If return_all is False: It returns the trained model (sklearn model), 
            the predictive accuracy of the test (numerical type),
            and the confusion matrix (numpy ndarray).
            >> If return_all is True: It returns the trained model (sklearn model), 
            the features training and test datasets (both numpy ndarrays), 
            the target training and test datasets (both numpy ndarrays),
            the predictive accuracy of the test (numerical type), and the 
            confusion matrix (numpy ndarray).
        '''
        trainedModel, features_train, features_test, target_train, target_test=train(df, featureColLabels=featureColLabels,
                                                        targetColLabel=targetColLabel,
                                                        model=model, train_size=train_size, 
                                                        random_state=random_state, shuffle=shuffle, return_all=True)
    
        (accuracy, precision, confusionMatrix)=test(features_test, target_test, trainedModel)
        
        if return_all is True:
            result = (trainedModel, features_train, features_test, target_train, 
                      target_test, accuracy, precision, confusionMatrix)
        elif return_all is False:
            result = (trainedModel, accuracy, precision, confusionMatrix)
        else:
            result = None
        return result

In [8]:
def plotConfusionMatrix(confusionMatrix, classes, title='Confusion matrix', 
                        cmap=None, figsize=None, ylabel='True classes', xlabel='Predicted classes',
                        save=False, filepath="confusion_matrix.png"):

    if cmap is None: cmap=pyplot.cm.Reds
    if figsize is None: figsize=[7,5]

    if not ((confusionMatrix.sum(axis=1)<=1).all()==True):
        # a test dataset of only 1 datapoint will fool the above condition.
        confusionMatrix=normalizeConfusionMatrix(confusionMatrix)

    confusionMatrix=confusionMatrix*100

    fig=pyplot.figure(figsize=figsize)
    ax=fig.add_subplot(1,1,1)

    axImg=ax.imshow(confusionMatrix, cmap=cmap)
    fig.colorbar(axImg, fraction=0.046, pad=0.04)

    ax.set_title(title)

    tick_positions = np.arange(len(classes))
    ax.set_xticks(tick_positions)
    ax.set_xticklabels(classes, rotation=45)
    ax.set_yticks(tick_positions)
    ax.set_yticklabels(classes)

    fmt = '.2f'
    threshold = (np.nanmax(confusionMatrix)+np.nanmin(confusionMatrix)) / 1.5
    r=confusionMatrix.shape[0]
    c=confusionMatrix.shape[1]

    for i, j in itertools.product(range(r), range(c)):
        ax.text(j, i, format(confusionMatrix[i, j], fmt)+"%",
                 horizontalalignment="center",
                 color="white" if (confusionMatrix[i, j] > threshold) else "black")

    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    
    fig.tight_layout()

    if save is True: fig.savefig(filepath)

    fig.show()

    return None

## Learning executions

In [9]:
lda=discriminant_analysis.LinearDiscriminantAnalysis(solver="svd", shrinkage=None, priors=None, 
                                                         n_components=None, store_covariance=False, tol=0.0001)

In [20]:
suppVM=svm.SVC(C=1000.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=2, shrinking=True,
  tol=0.001, verbose=False)

In [11]:
logit=linear_model.LogisticRegression(penalty="l2", dual=False, tol=0.0001, 
                                        C=1.0, fit_intercept=True, intercept_scaling=1, solver="saga", max_iter=100, 
                                        multi_class="multinomial", verbose=0, warm_start=False, n_jobs=None)

In [12]:
k=int(math.sqrt(len(imgData.index)))
k=k if (not k%2) else k+1

knn=neighbors.KNeighborsClassifier(n_neighbors=k, weights='uniform')

In [13]:
gnb=naive_bayes.GaussianNB(priors=None, var_smoothing=1e-09)

In [14]:
randForest=ensemble.RandomForestClassifier(n_estimators=10, criterion="gini", max_depth=None, min_samples_split=2,
                       min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features="auto", max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, 
                       n_jobs=None, random_state=2, verbose=0, warm_start=False, class_weight=None)

In [21]:
# myModels = [lda,suppVM,logit,knn,gnb,randForest]
myModels = [suppVM]

In [ ]:
tModels=[]
for myModel in myModels:
    (tModel, acc, p, cm)=trainAndTest(imgData, my_features_Column_labels, my_target_Column_label, model=myModel,
                                       train_size=0.8, random_state=2,shuffle=True, return_all=False, normalize_cm=True)
    tModels.append(tModel)
    print(tModel)
    print(acc)
    print(p)
    plotConfusionMatrix(cm, classes=["female", "male"])

In [17]:
### !!!!! SAVE myModel before closing!